In [ ]:
!pip install plotly

In [ ]:
import pandas as pd
import numpy as np
import warnings
import plotly.express as px
warnings.filterwarnings("ignore")
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle

In [ ]:
data = pd.read_csv("/content/heart.csv")

In [ ]:
data.head(2)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1


In [ ]:
data.shape

(918, 12)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [ ]:
data.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [ ]:
data.describe(include = "all")

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,918.000000,918,918,918.000000,918.000000,918.000000,918,918.000000,918,918.000000,918,918.000000
unique,NaN,2,4,NaN,NaN,NaN,3,NaN,2,NaN,3,NaN
top,NaN,M,ASY,NaN,NaN,NaN,Normal,NaN,N,NaN,Flat,NaN
freq,NaN,725,496,NaN,NaN,NaN,552,NaN,547,NaN,460,NaN
mean,53.510893,NaN,NaN,132.396514,198.799564,0.233115,NaN,136.809368,NaN,0.887364,NaN,0.553377
std,9.432617,NaN,NaN,18.514154,109.384145,0.423046,NaN,25.460334,NaN,1.066570,NaN,0.497414
min,28.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,60.000000,NaN,-2.600000,NaN,0.000000
25%,47.000000,NaN,NaN,120.000000,173.250000,0.000000,NaN,120.000000,NaN,0.000000,NaN,0.000000
50%,54.000000,NaN,NaN,130.000000,223.000000,0.000000,NaN,138.000000,NaN,0.600000,NaN,1.000000
75%,60.000000,NaN,NaN,140.000000,267.000000,0.000000,NaN,156.000000,NaN,1.500000,NaN,1.000000


In [ ]:
data.isnull().sum()

,0
Age,0
Sex,0
ChestPainType,0
RestingBP,0
Cholesterol,0
FastingBS,0
RestingECG,0
MaxHR,0
ExerciseAngina,0
Oldpeak,0


In [ ]:
data.duplicated().sum()

np.int64(0)

In [ ]:
data.nunique()

,0
Age,50
Sex,2
ChestPainType,4
RestingBP,67
Cholesterol,222
FastingBS,2
RestingECG,3
MaxHR,119
ExerciseAngina,2
Oldpeak,53


In [ ]:
categorical_columns = data.select_dtypes(include = "object").columns

In [ ]:
categorical_columns

Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object')

In [ ]:
for col in categorical_columns:
  print(col, (data[col].unique()), list(range(data[col].nunique())))
  print()
  data[col].replace((data[col].unique()), range(data[col].nunique()), inplace = True)

Sex ['M' 'F'] [0, 1]

ChestPainType ['ATA' 'NAP' 'ASY' 'TA'] [0, 1, 2, 3]

RestingECG ['Normal' 'ST' 'LVH'] [0, 1, 2]

ExerciseAngina ['N' 'Y'] [0, 1]

ST_Slope ['Up' 'Flat' 'Down'] [0, 1, 2]



In [ ]:
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,0,140,289,0,0,172,0,0.0,0,0
1,49,1,1,160,180,0,0,156,0,1.0,1,1
2,37,0,0,130,283,0,1,98,0,0.0,0,0
3,48,1,2,138,214,0,0,108,1,1.5,1,1
4,54,0,1,150,195,0,0,122,0,0.0,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    int64  
 2   ChestPainType   918 non-null    int64  
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    int64  
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    int64  
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    int64  
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(11)
memory usage: 86.2 KB


In [ ]:
data["Cholesterol"].value_counts()

,count
Cholesterol,
0,172
254,11
220,10
223,10
204,9
...,...
353,1
278,1
157,1


In [ ]:
data["Cholesterol"].mean()

np.float64(198.7995642701525)

In [ ]:
data["Cholesterol"].median()

223.0

In [ ]:
data["Cholesterol"].replace(0, np.nan, inplace =True)

In [ ]:
data["Cholesterol"].isna().sum()

np.int64(172)

In [ ]:
imputer = KNNImputer(n_neighbors = 3)
after_impute = imputer.fit_transform(data)
data = pd.DataFrame(after_impute, columns = data.columns)

In [ ]:
data["Cholesterol"].isna().sum()

np.int64(0)

In [ ]:
data.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [ ]:
data["RestingBP"][data["RestingBP"] == 0]

,RestingBP
449,0.0


In [ ]:
data["RestingBP"].replace(0, np.nan, inplace = True)

In [ ]:
imputer = KNNImputer(n_neighbors = 3)
after_impute = imputer.fit_transform(data)
data = pd.DataFrame(after_impute, columns = data.columns)

In [ ]:
data["RestingBP"].unique()

array([140., 160., 130., 138., 150., 120., 110., 136., 115., 100., 124.,
       113., 125., 145., 112., 132., 118., 170., 142., 190., 135., 180.,
       108., 155., 128., 106.,  92., 200., 122.,  98., 105., 133.,  95.,
        80., 137., 185., 165., 126., 152., 116., 144., 154., 134., 104.,
       139., 131., 141., 178., 146., 158., 123., 102.,  96., 143., 172.,
       156., 114., 127., 101., 174.,  94., 148., 117., 192., 129., 164.])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    float64
 1   Sex             918 non-null    float64
 2   ChestPainType   918 non-null    float64
 3   RestingBP       918 non-null    float64
 4   Cholesterol     918 non-null    float64
 5   FastingBS       918 non-null    float64
 6   RestingECG      918 non-null    float64
 7   MaxHR           918 non-null    float64
 8   ExerciseAngina  918 non-null    float64
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    float64
 11  HeartDisease    918 non-null    float64
dtypes: float64(12)
memory usage: 86.2 KB


In [ ]:
withoutoldpeak = data.columns
withoutoldpeak = withoutoldpeak.drop("Oldpeak")
data[withoutoldpeak] = data[withoutoldpeak].astype("int32")

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int32  
 1   Sex             918 non-null    int32  
 2   ChestPainType   918 non-null    int32  
 3   RestingBP       918 non-null    int32  
 4   Cholesterol     918 non-null    int32  
 5   FastingBS       918 non-null    int32  
 6   RestingECG      918 non-null    int32  
 7   MaxHR           918 non-null    int32  
 8   ExerciseAngina  918 non-null    int32  
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    int32  
 11  HeartDisease    918 non-null    int32  
dtypes: float64(1), int32(11)
memory usage: 46.7 KB


In [ ]:
data.corr()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
Age,1.000000,-0.055750,0.214164,0.263128,0.057108,0.198039,0.213152,-0.382045,0.215793,0.258612,0.268264,0.282039
Sex,-0.055750,1.000000,-0.179766,-0.009658,0.080395,-0.120076,0.018343,0.189186,-0.190664,-0.105734,-0.150693,-0.305445
ChestPainType,0.214164,-0.179766,1.000000,0.073016,0.038855,0.155111,0.112067,-0.273076,0.324049,0.300846,0.352364,0.459017
RestingBP,0.263128,-0.009658,0.073016,1.000000,0.113194,0.067556,0.094144,-0.109329,0.152621,0.174464,0.082401,0.118333
Cholesterol,0.057108,0.080395,0.038855,0.113194,1.000000,0.058798,0.074925,-0.028855,0.083761,0.065988,0.071345,0.101263
FastingBS,0.198039,-0.120076,0.155111,0.067556,0.058798,1.000000,0.050707,-0.131438,0.060451,0.052698,0.175774,0.267291
RestingECG,0.213152,0.018343,0.112067,0.094144,0.074925,0.050707,1.000000,0.048552,0.036119,0.114428,0.078807,0.061011
MaxHR,-0.382045,0.189186,-0.273076,-0.109329,-0.028855,-0.131438,0.048552,1.000000,-0.370425,-0.160691,-0.343419,-0.400421
ExerciseAngina,0.215793,-0.190664,0.324049,0.152621,0.083761,0.060451,0.036119,-0.370425,1.000000,0.408752,0.428706,0.494282
Oldpeak,0.258612,-0.105734,0.300846,0.174464,0.065988,0.052698,0.114428,-0.160691,0.408752,1.000000,0.501921,0.403951


In [ ]:
px.line(data.corr()["HeartDisease"][:-1].sort_values())

In [ ]:
px.sunburst(data, path=["HeartDisease", "Age"])

In [ ]:
px.histogram(data, x = "Age", color = "HeartDisease")

In [ ]:
px.pie(data, names = "HeartDisease", title = "Percentage of HeartDisease Classes Distribution")

In [ ]:
px.histogram(data, x = "Sex", color = "HeartDisease")

In [ ]:
px.histogram(data, x = "ChestPainType", color = "HeartDisease")

In [ ]:
px.sunburst(data, path=["HeartDisease", "RestingBP"])

In [ ]:
px.histogram(data, x = "FastingBS", color = "HeartDisease")

In [ ]:
px.sunburst(data, path = ["HeartDisease", "MaxHR"])

In [ ]:
px.violin(data, x = "HeartDisease", y = "MaxHR", color = "HeartDisease")

In [ ]:
px.violin(data, x = "HeartDisease", y = "Oldpeak", color = "HeartDisease")

In [ ]:
px.histogram(data, x = "ST_Slope", color = "HeartDisease")

In [ ]:
px.histogram(data, x = "ExerciseAngina", color = "HeartDisease")

In [ ]:
X = data.drop("HeartDisease", axis = 1)
y = data["HeartDisease"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [ ]:
X_train.shape

(734, 11)

In [ ]:
X_test.shape

(184, 11)

In [ ]:
y_train.shape

(734,)

In [ ]:
y_test.shape

(184,)

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, y_train)

LogisticRegression()

In [ ]:
lr_predict = lr.predict(X_test)
print(f"Logistic Regression Accuracy Score: {accuracy_score(y_test, lr_predict)}")

Logistic Regression Accuracy Score: 0.8532608695652174


In [ ]:
file = open("LogisticR.pkl", "wb")
pickle.dump(lr,file)

In [ ]:
clf = DecisionTreeClassifier()
param_grid = {
    "max_depth": [3,4,5,6,7,8],
    "min_samples_split": [2,3,4],
    "min_samples_leaf": [1,2,3],
    "random_state": [0, 42]
}

In [ ]:
grid_search = GridSearchCV(clf, param_grid, cv = 5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3, 4],
                         'random_state': [0, 42]})

In [ ]:
grid_search.best_params_

{'max_depth': 4,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'random_state': 0}

In [ ]:
dtc = DecisionTreeClassifier(**grid_search.best_params_)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=4, min_samples_leaf=2, random_state=0)

In [ ]:
dtc_pred = dtc.predict(X_test)
print("Decision Tree Accuracy Score : ", accuracy_score(y_test, dtc_pred))

Decision Tree Accuracy Score :  0.8043478260869565


In [ ]:
file = open("DT.pickle", "wb")
pickle.dump(dtc, file)

In [ ]:
rfc = RandomForestClassifier()
param_grid = {
    "n_estimators": [50, 100, 150, 200],
    "max_features": ["sqrt", "log2", None],
    "max_depth": [3,6,9,19],
    "max_leaf_nodes": [3,6,9]
}

In [ ]:
grid_search = GridSearchCV(rfc, param_grid)
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 6, 9, 19],
                         'max_features': ['sqrt', 'log2', None],
                         'max_leaf_nodes': [3, 6, 9],
                         'n_estimators': [50, 100, 150, 200]})

In [ ]:
grid_search.best_params_

{'max_depth': 6,
 'max_features': 'sqrt',
 'max_leaf_nodes': 9,
 'n_estimators': 150}

In [ ]:
rfclassifier = RandomForestClassifier(**grid_search.best_params_)
rfclassifier.fit(X_train, y_train)
rfclassifier_pred = rfclassifier.predict(X_test)
print("RandomForestClassifier Accuracy Score : ", accuracy_score(y_test, rfclassifier_pred))

RandomForestClassifier Accuracy Score :  0.8586956521739131


In [ ]:
file = open("RF.pickle", "wb")
pickle.dump(rfclassifier, file)